# Experiment
## Overview
In this experiment, we will use PPO algorithm to solve the classic CartPole task in Gym.

## Experiment
To conduct this experiment, we need the following building blocks.
- two vectorised environments, one for training and one for evaluation
- a PPO agent
- a replay buffer to store transition data
- two colectors to manage the data collecting process, one for training and one for evaluation
- a trainer to manage the training loop

### Preparation

In [1]:
import gymnasium as gym
import numpy as np
import torch

from tianshou.data import Collector, VectorReplayBuffer
from tianshou.env import DummyVectorEnv
from tianshou.policy import PPOPolicy
from tianshou.trainer import onpolicy_trainer
from tianshou.utils.net.common import ActorCritic, Net
from tianshou.utils.net.discrete import Actor, Critic

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'

### Environment
We create two vectorized environments both for training and testing. Since the execution time of CartPole is extremely short, there is no need to use multi-process wrappers and we simply use DummyVectorEnv.

In [2]:
env = gym.make('CartPole-v0')
train_envs = DummyVectorEnv([lambda: gym.make('CartPole-v0') for _ in range(20)])
test_envs = DummyVectorEnv([lambda: gym.make('CartPole-v0') for _ in range(10)])

### Policy
Next we need to initialise out PPO policy, PPO is an actor-critic-style on-policy algorithm, so we have to deine the actor and the critic in PPO first.

The actor is a neural network taht shares the same network head with the critic. Both networks' input is the environment observation. The output of the actor is the action and the output of the critic is a single value, representing the value of the current policy.

Luckily, Tianshou already provides basic network modules that we can use in this experiment.

In [3]:
# net is the shared head of the actor and the critic
net = Net(env.observation_space.shape, hidden_sizes=[64, 64], device=device)
actor = Actor(net, env.action_space.n, device=device).to(device)
critic = Critic(net, device=device).to(device)
actor_critic = ActorCritic(actor, critic)

# optimiser of the actor and the critic
optim = torch.optim.Adam(actor_critic.parameters(), lr=0.0003)

Once we have defined the actor, the critic and the optimiser. We can use them to construct our PPO agent. CartPole is a discrete action space problem, so the distribution of our action space can be a categorical distribution.

In [4]:
dist = torch.distributions.Categorical
policy = PPOPolicy(actor, critic, optim, dist, action_space=env.action_space, deterministic_eval=True)

`deterministic_eval=True` means that we want to sample actions during training but we would like to always use the best action in evaluation. No randomness included.

### Collector
We can set up the collectors now. Train collector is used to collect and store training data, so an additional replay buffer has to be passed in.

In [5]:
train_collector = Collector(policy, train_envs, VectorReplayBuffer(2000, len(train_envs)))
test_collector = Collector(policy, test_envs)

We use `VectorReplayBuffer` here because it's more efficient to collaborate with vectorised environments, you can simply consider `VectorReplayBuffer` as a list of ordinary replay buffers.

### Trainer
Finally, we can use the trainer to help us set up the training loop.

In [6]:
result = onpolicy_trainer(
    policy,
    train_collector,
    test_collector,
    max_epoch=10,
    step_per_epoch=50000,
    repeat_per_collect=10,
    episode_per_test=10,
    batch_size=256,
    step_per_collect=2000,
    stop_fn=lambda mean_reward: mean_reward >= 195
)

Epoch #1: 50001it [00:08, 5590.06it/s, env_step=50000, len=119, loss=55.307, loss/clip=0.003, loss/ent=0.595, loss/vf=110.618, n/ep=8, n/st=2000, rew=119.88]                             

Epoch #1: test_reward: 200.000000 ± 0.000000, best_reward: 200.000000 ± 0.000000 in #1


### Results
Print the training result.

In [7]:
print(result)

{'duration': '9.15s', 'train_time/model': '5.54s', 'test_step': 2089, 'test_episode': 20, 'test_time': '0.18s', 'test_speed': '11330.30 step/s', 'best_reward': 200.0, 'best_result': '200.00 ± 0.00', 'train_step': 50000, 'train_episode': 1117, 'train_time/collector': '3.43s', 'train_speed': '5574.81 step/s'}


We can also test our trained agent.

In [8]:
policy.eval()
result = test_collector.collect(n_episode=1, render=False)
print("Final reward: {}, length: {}".format(result["rews"].mean(), result["lens"].mean()))

Final reward: 200.0, length: 200.0
